<a href="https://colab.research.google.com/github/debbysonino/LamasDataHack/blob/master/DataLearn_2019_Scaffold_basefilesecondprediction0.45.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

&nbsp; ![alt text](https://s3.amazonaws.com/monday.com/static/svg/monday-logos/monday-footer-logo.svg)

#Model scaffold
This notebook is intended to get you up and running faster.

It has the basic scaffold of an ML model, including:
* Data loading
* Feature extraction
* Columns transformation
* Training
* Evaluating
* Submitting results

###Getting our depnedncies (and data!)
First we'll import our relevant libraries

In [0]:
# General DS libraries we are going to need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Importing our base model
# [REDACTED ML MODEL USED]

# Imports for working with our large dataset
from sklearn.utils.random import sample_without_replacement
from sklearn.model_selection import train_test_split

# We need those for data manipulation and getting our features ready for the model
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer

# These can be used to measure our model's performance
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# Ignore DataFrame assignment warnings
pd.options.mode.chained_assignment = None


We set a few constants to use later on for sampeling and running the model

In [0]:
#@title Model parameters { run: "auto" }
# n_neighbors = 7 #@param {type:"slider", min:1, max:30, step:1}
group_name = "Lamassim" #@param {type:"string"}
samples_num = 230000 #@param {type:"slider", min:0, max:1500000, step:10000}
n_jobs = -1 #@param {type:"slider", min:-1, max:32, step:1}
path_prefix = "https://storage.googleapis.com/mondaycom-datahack/final_sets" #@param ["https://storage.googleapis.com/mondaycom-datahack/final_sets", "https://mondaycom-datahack.s3.amazonaws.com/final_sets"] {allow-input: true}

Next we'll load all the different parts of our dataset

<br/>

_Our use my data loading [snippet](https://colab.research.google.com/drive/1_Y-sZ5eHIDlDUMuLCwfnbuJdLh0DTXmO#scrollTo=5HGlaJTEAYJu&line=23&uniqifier=1)!_

In [3]:
import os
import pandas as pd

# We define the datasets we want to load
datasets = ('accounts', 'users', 'events', 'subscriptions')
source_prefix = 'https://storage.googleapis.com/mondaycom-datahack/final_sets/'

local_dir = './datasets/datahack/'
file_prefix = 'train_'
file_suffix = ''
file_extension = 'csv'

# We create a directory for the datasets if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

# For each dataset we want, we check if we already downloaded it and fix it if we didn't
for dataset in datasets:
  if not os.path.isfile('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension)):
    !curl {source_prefix}{file_prefix}{dataset}{file_suffix}.{file_extension} --output {local_dir}{file_prefix}{dataset}{file_suffix}.{file_extension}

  # Load the datasets into a DataFrame using pandas
  globals()['{}{}'.format(file_prefix, dataset)] = pd.read_csv('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension), low_memory=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  204M  100  204M    0     0  32.1M      0  0:00:06  0:00:06 --:--:-- 53.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  330M  100  330M    0     0   279M      0  0:00:01  0:00:01 --:--:--  279M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1215M  100 1215M    0     0   170M      0  0:00:07  0:00:07 --:--:--  183M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.1M  100 17.1M    0     0   132M      0 --:--:-- --:--:-- --:--:--  132M


In [4]:
import os
import pandas as pd

# We define the datasets we want to load
datasets = ('accounts', 'users', 'events', 'subscriptions')
source_prefix = 'https://storage.googleapis.com/mondaycom-datahack/final_sets/'

local_dir = './datasets/datahack/'
file_prefix = 'test_'
file_suffix = ''
file_extension = 'csv'

# We create a directory for the datasets if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

# For each dataset we want, we check if we already downloaded it and fix it if we didn't
for dataset in datasets:
  if not os.path.isfile('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension)):
    !curl {source_prefix}{file_prefix}{dataset}{file_suffix}.{file_extension} --output {local_dir}{file_prefix}{dataset}{file_suffix}.{file_extension}

  # Load the datasets into a DataFrame using pandas
  globals()['{}{}'.format(file_prefix, dataset)] = pd.read_csv('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension), low_memory=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6M  100 10.6M    0     0  68.6M      0 --:--:-- --:--:-- --:--:-- 68.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.1M  100 17.1M    0     0   100M      0 --:--:-- --:--:-- --:--:--  100M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.2M  100 63.2M    0     0   118M      0 --:--:-- --:--:-- --:--:--  118M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  959k  100  959k    0     0  27.5M      0 --:--:-- --:--:-- --:--:-- 27.5M


We need to add our test sets to our train sets and work on both at the same time.

We'll split them back up before training and inference.

In [5]:
accounts = train_accounts.append(test_accounts)
users = train_users.append(test_users)
events = train_events.append(test_events)
subscriptions = train_subscriptions.append(test_subscriptions)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


###Feature engineering
In this block we add a new feature of `[REDACTED]` extracted from the user `[REDACTED]`

We also seperate all the `[REDACTED]` users into a different DataFrame

Let's enrich our data a bit

In [0]:
accounts["numuser"]=users.groupby("account_id").user_id.size()
accounts["numisadmin"]=users.groupby("account_id").is_admin.sum()
accounts["numenabled"]=users.groupby("account_id").enabled.sum()
accounts["numpending"]=users.groupby("account_id").pending.sum()
accounts["numcountry"]=users.groupby("account_id").country.count()
accounts["numregion"]=users.groupby("account_id").region.count()
accounts["numrcity"]=users.groupby("account_id").city.count()
accounts["numphoto"]=users.groupby("account_id").has_photo.sum()
accounts["nummos"]=users.groupby("account_id").os.count()
accounts["nummobile"]=users.groupby("account_id").device.count()
accounts["nummobile2"]=users[users["device"]=="mobile"].groupby("account_id").device.count()
accounts["numchrome"]=users[users["browser"]=="chrome"].groupby("account_id").browser.count()
#accounts["vetekuser"]=users.groupby("account_id").created_at.min()
#accounts["vetekactive"]=users.groupby("account_id").became_active_at.min()


#accounts["vetekuser"]=users.groupby("account_id").created_at.min()
#accounts["vetekactive"]=users.groupby("account_id").became_active_at.min()



In [0]:
#sub_df = pd.read_csv('https://storage.googleapis.com/mondaycom-datahack/final_sets/train_subscriptions.csv', nrows=7000000000)
def upgrade(row):
  return abs(row["plan_id"]-row["prev_plan_id"])>0


subscriptions["plan_upgrade"] =  subscriptions.apply(upgrade ,axis=1)

subscriptions['status_success']=subscriptions['status'].fillna('SUCCESS')
subscriptions['status_success']=subscriptions['status_success']=='SUCCESS'
#sub_df[sub_df['status_success']==0].sum() # check if mrr=0 for status=failed

account_subs=subscriptions.groupby('account_id').agg({"subscription_id":"count","plan_id":"nunique", "plan_upgrade": "mean","mrr_gain":"sum"})
account_subs.rename(columns={'subscription_id':'sub_count',
                          'plan_id':'unique_plan',
                          
                             'mrr_gain':'mrr_gain_sum'}, 
                 inplace=True)
accounts=pd.merge(accounts, account_subs, on='account_id', how='left')
#events_by_account_user = events.groupby(['account_id'])['payment_events'].sum()
accounts['browser'] = accounts['browser'].fillna('chrome')
accounts['churn_reason'] = accounts['churn_reason'].fillna(0)
accounts['payment_events_by_account']=events.groupby(['account_id'])['payment_events'].sum()
accounts['payment_events_by_account'] = accounts['payment_events_by_account'].fillna(0)
accounts['last_event_day']=events.groupby(['account_id'])['date'].max()
# accounts['last_event_day'].fillna(accounts['subscription_started_at'])
# from the resume data
accounts['Avg_event_by_total']=(events.groupby(['account_id']).total_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_column']=(events.groupby(['account_id']).column_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_board']=(events.groupby(['account_id']).board_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_num']=(events.groupby(['account_id']).num_of_boards.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_count']=(events.groupby(['account_id']).count_kind_columns.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_content']=(events.groupby(['account_id']).content_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_group']=(events.groupby(['account_id']).group_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_invite']=(events.groupby(['account_id']).invite_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_import']=(events.groupby(['account_id']).import_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_notification']=(events.groupby(['account_id']).notification_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_new']=(events.groupby(['account_id']).new_entry_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_payment']=(events.groupby(['account_id']).payment_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_inbox']=(events.groupby(['account_id']).inbox_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_communicating']=(events.groupby(['account_id']).communicating_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_non']=(events.groupby(['account_id']).non_communicating_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_web']=(events.groupby(['account_id']).web_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_ios']=(events.groupby(['account_id']).ios_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_android']=(events.groupby(['account_id']).android_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_desktop']=(events.groupby(['account_id']).desktop_app_events.sum()/events.groupby(['account_id']).date.count())
accounts['Avg_event_by_empty']=(events.groupby(['account_id']).empty_events.sum()/events.groupby(['account_id']).date.count())


accounts['P_event_by_column']=events.groupby(['account_id']).column_events.sum()/(events.groupby(['account_id']).total_events.sum())                                                                   
accounts['P_event_by_board']=events.groupby(['account_id'])['board_events'].sum()/(events.groupby(['account_id']).total_events.sum())                                                 
accounts['P_event_by_num']=events.groupby(['account_id']).num_of_boards.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_count']=events.groupby(['account_id']).count_kind_columns.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_content']=events.groupby(['account_id']).content_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_group']=events.groupby(['account_id']).group_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_invite']=events.groupby(['account_id']).invite_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_import']=events.groupby(['account_id']).import_events.sum()/(events.groupby(['account_id']).total_events.sum())                                                                
accounts['P_event_by_notification']=events.groupby(['account_id']).notification_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_new']=events.groupby(['account_id']).new_entry_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_payment']=events.groupby(['account_id']).payment_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_inbox']=events.groupby(['account_id']).inbox_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_communicating']=events.groupby(['account_id']).communicating_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_non']=events.groupby(['account_id']).non_communicating_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_web']=events.groupby(['account_id']).web_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_ios']=events.groupby(['account_id']).ios_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_android']=events.groupby(['account_id']).android_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_desktop']=events.groupby(['account_id']).desktop_app_events.sum()/(events.groupby(['account_id']).total_events.sum())
accounts['P_event_by_empty']=events.groupby(['account_id']).empty_events.sum()/(events.groupby(['account_id']).total_events.sum())



In [0]:
popular_emails = pd.read_csv('https://raw.githubusercontent.com/debbysonino/LamasDataHack/master/popular_emails.csv', nrows=7000000000)
users['email'].value_counts()
users['domain']=users['email'].apply(lambda x: x.split('@')[-1])
users['mailsuffix']=users['domain'].apply(lambda x: x.split('.')[1])
users['domain_indic']=users['domain'].isin(popular_emails)
accounts['domain']=1-users.groupby('account_id').domain_indic.mean()

In [0]:
accounts['time_between_created_trail'] = (pd.to_datetime(accounts['trial_start']) - pd.to_datetime(accounts['created_at'])).apply(lambda x: (x.seconds//3600))


In [0]:
import time
import datetime
accounts['day_start']=accounts['created_at'].apply(lambda x: pd.to_datetime(x).strftime("%A"))
accounts['is_weekend']=accounts['day_start'].isin(['Saturday','Sunday'])

In [0]:
accounts['sum_event_by_account']=(events.groupby(['account_id']).total_events.sum())

AttributeError: ignored

In [0]:
all_features=accounts

In [309]:
subscriptions.columns

Index(['event_happened_at', 'subscription_id', 'account_id', 'plan_id',
       'event_type', 'invoice_charge_amount', 'prev_plan_id', 'status',
       'status_reason', 'currency', 'invoice_charge_amount_usd', 'mrr_gain',
       'next_charge_date', 'payment_type', 'transaction_date', 'plan_upgrade',
       'status_success'],
      dtype='object')

###Data preperation
After we created our raw features we need to make sure the fit the way our ML model expects to receive them.

###Re-splitting
We now need to split our data back to the original train set and test set.

We also make sure we keep only the columns we want in the data frame (the features)

###Setting everything up
Our dataset is large (1,500,000+ accounts, each has a few users, each has events for every day)

We need to work on a smaller batch of the training data so we can iterate more quickly.

Once we find a good architecture we can increase the sample size to increase the accuracy.

In [0]:
# We map our features into different types
categorical_features = ['country' , 'device','industry','os','user_goal', 'churn_reason' ]
# status_success

normalized_features = [ 'company_size', 'max_team_size','mrr',
                       "numuser",   "numenabled", 
                    "numrcity",  "numphoto", "nummos", "nummobile", 'time_diff',  "mrr_gain_sum", 
                       'payment_events_by_account', 'domain'
                      ]



binary_features = ['paying', 'has_logo', "plan_upgrade"]

untouched_features = ['account_id']

target = ['lead_score']

# And create a column transformer to handle the manipulation for us
preprocess = make_column_transformer(
    (OneHotEncoder(), categorical_features+['plan_id']),
    (Normalizer(), normalized_features),
    (Binarizer(), binary_features)
)
# Getting only the relevant features from the dataset
dataset = all_features[categorical_features + normalized_features + binary_features + ['plan_id'] + untouched_features + target]

# Filling empty values with default values 
dataset.loc[:,categorical_features] = dataset[categorical_features].fillna('')
dataset.loc[:,'plan_id'] = dataset['plan_id'].fillna(0)

dataset.loc[:,normalized_features +
              binary_features +
              untouched_features] = dataset[normalized_features +
                                            binary_features +
                                            untouched_features].fillna(0)

# Splitting them back up to the original train/test split
dataset_train = dataset[dataset.reset_index().account_id.isin(train_accounts.account_id)]
dataset_test = dataset[dataset.reset_index().account_id.isin(test_accounts.account_id)]
# Splitting them back up to the original train/test split
dataset_train = dataset.loc[~(dataset['lead_score'].isna())].reset_index().drop(['index'],axis=1)
dataset_test = dataset.loc[(dataset['lead_score'].isna())].reset_index().drop(['index'],axis=1)
sampled_dataset_train = dataset_train.iloc[sample_without_replacement(dataset_train.shape[0], samples_num)]
# We fit our column transformer on both the train and the test sets
preprocess.fit(sampled_dataset_train.append(dataset_test))

# We use transform to finally manipulate the features of our training set
X = preprocess.transform(sampled_dataset_train)

# Seperating the label
y = sampled_dataset_train.pop('lead_score')

In [0]:
sampled_dataset_train = dataset_train.iloc[sample_without_replacement(dataset_train.shape[0], samples_num)]
# We fit our column transformer on both the train and the test sets
preprocess.fit(sampled_dataset_train.append(dataset_test))

# We use transform to finally manipulate the features of our training set
X = preprocess.transform(sampled_dataset_train)

# Seperating the label
y = sampled_dataset_train.pop('lead_score')

In [337]:
y.mean()

0.024665217391304346

In [0]:
# You now need to split the data into YOUR OWN training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# For standardization purposes we store y_test in a y_true variable
y_true = y_test

In [339]:
## (when using google colab)
! pip install catboost
! pip install plotly_express

In [0]:
## General
import os 
import joblib
import requests
from google_drive_downloader import GoogleDriveDownloader as gdd

## Data manipulation
import pandas as pd
import numpy as np

## Modeling
### Modeling pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
 
### Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier    
from catboost import CatBoostClassifier, Pool

## Visuatlization
import plotly 
import plotly.express as px
import plotly.graph_objects as go

In [341]:
parameters = {'n_estimators': [50], #[50,100,200]
             'max_depth': [40], #[5,10,15],
             'criterion': ['gini', 'entropy'],
             'max_features': [20]} #[int(np.log2(X_train.shape[1])), int(np.sqrt(X_train.shape[1]))]

rf_cv = GridSearchCV(RandomForestClassifier(class_weight='balanced'), 
                   parameters, 
                   n_jobs = -1,
                   cv = 5,
                   refit = True,
                   scoring = 'f1')

rf_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                             

In [342]:
rf_best_model = rf_cv.best_estimator_
rf_best_model

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=40, max_features=20,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=50, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [0]:
y_train_pred = rf_best_model.predict(X_train) 

In [344]:
accuracy_score(y_train_pred, y_train),f1_score(y_train, y_train_pred),precision_score(y_train, y_train_pred),recall_score(y_train, y_train_pred)

(0.9762105263157895,
 0.6732461654513452,
 0.5089820359281437,
 0.9940597735288658)

In [345]:
confusion_matrix(y_train_pred, y_train)



array([[207947,     32],
       [  5166,   5355]])

## XGBOOST

###Running the model
It's the money time, we can finally run our model!

First we need to created it, and train(fit) it.

In [346]:
# Now we need to get the predictions of our test set
%time y_pred = rf_best_model.predict(X_test)

CPU times: user 103 ms, sys: 964 µs, total: 104 ms
Wall time: 103 ms


###Model evaluation
Now that we have our model and it can predict the lead score based on features, we need a way to test if it's any good

####Classification report
We use classification_report to get different metrics comparing our prediction to the ground truth.

In [347]:
print(classification_report(y_true, y_pred, target_names=['Not Lead', 'Lead']))

              precision    recall  f1-score   support

    Not Lead       0.99      0.97      0.98     11214
        Lead       0.33      0.62      0.43       286

    accuracy                           0.96     11500
   macro avg       0.66      0.79      0.70     11500
weighted avg       0.97      0.96      0.96     11500



In [348]:
importances = rf_best_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_best_model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

Feature ranking:


We can also get the MCC score of the model

In [349]:
print('Acc:  {}'.format(metrics.accuracy_score(y_true, y_pred)))
print('MCC: {}'.format(metrics.matthews_corrcoef(y_true, y_pred)))
print('F1:  {}'.format(metrics.f1_score(y_true, y_pred)))

Acc:  0.9586956521739131
MCC: 0.4321690382075961
F1:  0.42839951865222625


####Plotting the confusion matrix
Confusion matrices are useful for comparing our predictions

###Submitting results
After you ran several iterations, and you think your model is good enough, you can send it to us and we'll add your score on the leaderboard!

You have to get the results into the following format:
```python
{"9023749": 1, "9837598": 0, ...}
```

This is a dictionary where the keys are `account_id`s and the values are the predicted lead_score.

_Make sure you send us **all** the test accounts!_

_There should be exactly `71,683` of them!_

####Prediction
First of all, just like before, we have to predict the lead_score.

This time you need to use the test set _we_ provided.

In [0]:
dataset_test = dataset[dataset['lead_score'].isna()]
submission_account_ids = dataset_test.account_id

X_submission = preprocess.transform(dataset_test)

y_pred_submission = rf_best_model.predict(X_submission)

####Submission
Now that we have our submission predictions, we need to pack them up into a compatible format for our server to handle.


In [0]:
# Creating a dictionary where the keys are the account_ids
# and the values are your predictions
prediction = dict(zip(map(str, map(int,submission_account_ids)), map(int,y_pred_submission)))


We now just send the results to our server and wait for the score!

In [352]:
# Importing stuff for http requests
from urllib import request
import json

# We validate first that we actually send all the test accounts expected to be sent
if y_pred_submission.shape[0] != 71683 or submission_account_ids.shape[0] != 71683:
  raise Exception("You have to send all of the accounts! Expected: (71683, 71683), Got: ({}, {})".format(y_pred_submission.shape[0], submission_account_ids.shape[0]))

if "group_name" not in vars() or group_name == "":
  group_name = input("Please enter your group's name:")

data = json.dumps({'submitter': group_name, 'predictions': prediction}).encode('utf-8')

req = request.Request(f"https://leaderboard.datahack.org.il/monday/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)

res = request.urlopen(req)
print(json.load(res))

{'member': 'Lamassim', 'rank': 3, 'score': 0.4496973358884644}
